In [63]:
import pandas as pd

In [64]:
data_allocine = pd.read_csv("../scrapper/infosfilmsallocine.csv")
data_target_allocine = pd.read_csv("../scrapper/releaseFrUsaAllocine.csv")




In [65]:
# Remplacer les valeurs manquantes dans les colonnes 'actors' et 'director' par des chaînes vides
data_allocine['actors'] = data_allocine['actors'].fillna('')
data_allocine['director'] = data_allocine['director'].fillna('')

# Diviser les colonnes 'actors' en listes d'acteurs
data_allocine['actors'] = data_allocine['actors'].apply(lambda x: x.split(',')[1:] if isinstance(x, str) else [])

# Diviser la colonne 'director' et supprimer le préfixe "Par" et "De"
data_allocine['director'] = data_allocine['director'].apply(lambda x: x.replace('Par,', '').replace('De,', '') if isinstance(x, str) else '')

# Diviser la colonne 'director' en une liste
data_allocine['director'] = data_allocine['director'].apply(lambda x: x.split(',') if isinstance(x, str) else [])



In [66]:
# suppresion de colones:
col_to_drop = ["image_urls"]
data_allocine = data_allocine.drop(col_to_drop, axis = 1)

In [67]:
# Fusionner les datasets sur la colonne 'film_title' et 'title'
merged_data = pd.merge(data_allocine, data_target_allocine[['film_title', 'entrees_fr']], left_on='title', right_on='film_title', how='left')

# Supprimer la colonne 'film_title' redondante
merged_data.drop(columns=['film_title'], inplace=True)

# Afficher le résultat
display(merged_data)

,actors,director,nationalite,studio,timing,title,titre_original,entrees_fr
0,"[Audrey Lamy, Fabrice Eboué, Anouk Grinberg]","[Maxime Govare, Romain Choay, Maxime Govare, R...",France,Warner Bros. France,1h 43min,Heureux gagnants,NaN,136 628
1,"[Leonie Benesch, Michael Klammer, Rafael Stach...","[İlker Çatak, İlker Çatak, Johannes Duncker]",Allemagne,Tandem,1h 39min,La Salle des profs,Das Lehrerzimmer,90 612
2,"[Paola Cortellesi, Valerio Mastandrea, Romana ...","[Paola Cortellesi, Paola Cortellesi, Furio And...",Italie,Universal Pictures International France,1h 58min,Il reste encore demain,C'è ancora domani,145 340
3,"[Timothée Chalamet, Rebecca Ferguson, Oscar Is...","[Denis Villeneuve, Jon Spaihts, Denis Villeneuve]",U.S.A.,Warner Bros. France,2h 36min,Dune,NaN,1 030 687
4,"[Raphaël Personnaz, Doria Tillier, Jeanne Bali...","[Anne Fontaine, Anne Fontaine, Claire Barré]",France,SND,2h 00min,Bolero,NaN,177 724
...,...,...,...,...,...,...,...,...
14621,[],[Teddy Wayne],U.S.A.,NaN,NaN,The Winner,NaN,NaN
14622,"[Eletricer Kache Hamisi, Bibi Swaleh, Bosco Ba...","[Damien Hauser, Damien Hauser]",Kenya,NaN,1h 31min,After the Long Rains,Baada ya masika,NaN
14623,"[Kate Watson, Michael DeVorzon, Eric Roberts]","[Jared Cohn, Jared Cohn, Naomi L. Selfman]",U.S.A.,NaN,1h 30min,Fiancée à un imposteur,Her Deadly Groom,NaN
14624,"[Tom Everett Scott, Ella Ballentine, Mia Maestro]","[Peter Benson (IV), Melissa Salmons]",U.S.A.,NaN,1h 30min,Ensemble à tout prix!,"Rise and Shine, Benedict Stone",NaN


In [68]:
# Afficher le DataFrame et supprimer les lignes avec des valeurs vide

merged_data = merged_data.dropna()

display(merged_data)

,actors,director,nationalite,studio,timing,title,titre_original,entrees_fr
1,"[Leonie Benesch, Michael Klammer, Rafael Stach...","[İlker Çatak, İlker Çatak, Johannes Duncker]",Allemagne,Tandem,1h 39min,La Salle des profs,Das Lehrerzimmer,90 612
2,"[Paola Cortellesi, Valerio Mastandrea, Romana ...","[Paola Cortellesi, Paola Cortellesi, Furio And...",Italie,Universal Pictures International France,1h 58min,Il reste encore demain,C'è ancora domani,145 340
5,"[Timothée Chalamet, Zendaya, Rebecca Ferguson]","[Denis Villeneuve, Denis Villeneuve, Jon Spaihts]",U.S.A.,Warner Bros. France,2h 46min,Dune : Deuxième Partie,Dune: Part Two,1 383 999
7,"[Anthony Hopkins, Johnny Flynn, Helena Bonham ...","[James Hawes, Lucinda Coxon, Nick Drake]",Grande-Bretagne,SND,1h 49min,Une vie,One Life,456 390
12,"[Olivia Colman, Jessie Buckley, Anjana Vasan]","[Thea Sharrock, Jonny Sweet]",Grande-Bretagne,StudioCanal,1h 40min,Scandaleusement vôtre,Wicked Little Letters,51 921
...,...,...,...,...,...,...,...,...
4703,"[America Young, Greg Chun, Kirsten Day]","[Scott Pleydell-Pearce, Christopher Keenan, Ka...",U.S.A.,\n385 entrées\n,1h 00min,"Barbie : Grande Ville, Grands Rêves","Barbie: Big City, Big Dreams",391
4812,"[Lu Celeste, Wu Xiaoliang, Jiajia Wang]","[Liang Ming, Liang Ming]",Chine,ASC Distribution,1h 44min,Grand frère,Wisdom Tooth,414
4948,"[Leopold Pallua, Rosa Zant, Anna Fenderl]","[Arash T. Riahi, Arash T. Riahi]",Autriche,Les Films du Losange,1h 42min,Oskar et Lily,Ein bisschen bleiben wir noch,1 324
4988,"[Astrid Adverbe, Dimitra Kontou, Maritza Gonza...","[Pamela Varela, Pamela Varela]",France,Cinéma Saint-André des Arts,1h 28min,Le Voyage d'Ana,El Viaje de Ana,108


In [69]:
# Aplatir les colonnes contenant des listes d'acteurs
merged_data['num_actors'] = merged_data['actors'].apply(len)  # Nombre d'acteurs
merged_data['actor_1'] = merged_data['actors'].apply(lambda x: x[0] if len(x) >= 1 else None)
merged_data['actor_2'] = merged_data['actors'].apply(lambda x: x[1] if len(x) >= 2 else None)
merged_data['actor_3'] = merged_data['actors'].apply(lambda x: x[2] if len(x) >= 3 else None)
# merged_data['actor_4'] = merged_data['actors'].apply(lambda x: x[3] if len(x) >= 4 else None)

# Aplatir les colonnes contenant des listes de réalisateurs
merged_data['num_director'] = merged_data['director'].apply(len)  # Nombre de réalisateurs
merged_data['director_1'] = merged_data['director'].apply(lambda x: x[0] if len(x) >= 1 else None)
merged_data['director_2'] = merged_data['director'].apply(lambda x: x[1] if len(x) >= 2 else None)
merged_data['director_3'] = merged_data['director'].apply(lambda x: x[2] if len(x) >= 3 else None)
# merged_data['director_4'] = merged_data['director'].apply(lambda x: x[3] if len(x) >= 4 else None)

# Supprimer les colonnes originales avec des listes si nécessaire
merged_data.drop(columns=['actors', 'director'], inplace=True)


In [70]:
#suppresion de l'espace dans le nombre d'entree

merged_data['entrees_fr'] = merged_data['entrees_fr'].str.replace(' ', '').astype(int)



In [71]:
# Convertir le timing en minutes
def convert_timing(timing):
    timing_cleaned = timing.replace('min', '')  # Supprimer 'min' de la chaîne
    hours, minutes = map(int, timing_cleaned.split('h '))
    total_minutes = hours * 60 + minutes
    return total_minutes



merged_data['timing'] = merged_data['timing'].apply(convert_timing)


In [72]:
# # transformer les pays en nombre
# # display(merged_data)
# # {' Allemagne': 0, ' Italie': 1, ' U.S.A.': 2, ' Grande-Bretagne': 3, ' Irlande': 4, ' Chine': 5, ' Jordanie': 6, ' Iran': 7, ' Corée du Sud': 8, ' Pologne': 9, ' Chili': 10, ' Belgique': 11, ' Japon': 12, ' Espagne': 13, ' Finlande': 14, ' Danemark': 15, ' Grèce': 16, ' Turquie': 17, ' Suède': 18, ' Australie': 19, ' Mongolie': 20, ' Russie': 21, ' Yemen': 22, ' France': 23, ' Suisse': 24, ' Canada': 25, ' Indonésie': 26, ' Ukraine': 27, ' Norvège': 28, ' kazakhstan': 29, ' Soudan': 30, ' Argentine': 31, ' Géorgie': 32, ' Bhoutan': 33, ' Lituanie': 34, ' Brésil': 35, ' Roumanie': 36, ' Hong-Kong': 37, ' Egypte': 38, ' Bosnie-Herzégovine': 39, ' Tunisie': 40, ' Hongrie': 41, ' Guatemala': 42, ' Islande': 43, ' Slovaquie': 44, ' Pérou': 45, ' Portugal': 46, ' Pays-Bas': 47, ' Colombie': 48, ' Luxembourg': 49, ' Autriche': 50, ' Palestine': 51, ' République tchèque': 52, ' Macédoine': 53, ' Inde': 54, ' Philippines': 55, ' Kosovo': 56, ' Bolivie': 57, ' Israël': 58, ' Liban': 59, ' Taïwan': 60, ' Algérie': 61, ' Cambodge': 62, ' Uruguay': 63, ' Thaïlande': 64, ' Lésotho': 65, ' Costa Rica': 66, ' Mexique': 67}
# nationalities = merged_data['nationalite'].unique()
# nationality_dict = {nationality: i for i, nationality in enumerate(nationalities)}
# merged_data['nationalite'] = merged_data['nationalite'].map(nationality_dict)



In [73]:
#Donner une cote aux acteurs
from collections import Counter
import numpy as np

# Combiner les listes d'acteurs dans une seule liste
all_actors = merged_data['actor_1'].tolist() + merged_data['actor_2'].tolist() + merged_data['actor_3'].tolist()

# Utiliser Counter pour compter le nombre d'occurrences de chaque acteur
actor_counts = Counter([actor for actor in all_actors if isinstance(actor, str)])

# Trier les acteurs par nombre d'occurrences (le plus fréquent en premier)
sorted_actors = sorted(actor_counts.items(), key=lambda x: x[1], reverse=True)

# Créer un dictionnaire pour stocker les scores des acteurs
actor_scores = {}

# Assigner des scores en fonction du classement
for i, (actor, count) in enumerate(sorted_actors, start=1):
    actor_scores[actor] = i

print(actor_scores)
print(len(actor_scores))

# Définir une fonction pour mapper les acteurs à leur score de popularité
def map_actor_to_popularity(actor, actor_scores):
    if isinstance(actor, str) and actor in actor_scores:
        return actor_scores[actor]
    else:
        return np.nan

# Ajouter la fonctionnalité popularity_actor_1
merged_data['popularity_actor_1'] = merged_data['actor_1'].map(lambda x: map_actor_to_popularity(x, actor_scores))

# Ajouter la fonctionnalité popularity_actor_2
merged_data['popularity_actor_2'] = merged_data['actor_2'].map(lambda x: map_actor_to_popularity(x, actor_scores))

# Ajouter la fonctionnalité popularity_actor_3
merged_data['popularity_actor_3'] = merged_data['actor_3'].map(lambda x: map_actor_to_popularity(x, actor_scores))

# Afficher les premières lignes du DataFrame pour vérification
display(merged_data.head())

{'Pierfrancesco Favino': 1, 'Natsuki Hanae': 2, 'Penélope Cruz': 3, 'Sandra Hüller': 4, 'Chris Pratt': 5, 'Harris Dickinson': 6, 'Sam Rockwell': 7, "Josh O'Connor": 8, 'Paula Beer': 9, 'Léa Seydoux': 10, 'Kylian Trouillard': 11, 'Kim Min-Hee': 12, 'Antonio Banderas': 13, 'Michelle Rodriguez': 14, 'Samuel L. Jackson': 15, 'Emma Stone': 16, 'Kenneth Branagh': 17, 'Satoshi Tsumabuki': 18, 'Jason Statham': 19, 'Zac Efron': 20, 'Joaquin Phoenix': 21, 'Hae-hyo Kwon': 22, 'Alma Pöysti': 23, 'Willem Dafoe': 24, 'Sakura Andô': 25, 'Jason Momoa': 26, 'Shailene Woodley': 27, 'Ralph Fiennes': 28, 'Vin Diesel': 29, 'Mana Ashida': 30, 'Lesley Manville': 31, 'Angourie Rice': 32, 'Lévanah Solomon': 33, "Alexandre N'Guyen": 34, 'Tahar Rahim': 35, 'Adam Driver': 36, 'Leslie Odom Jr.': 37, 'Gerard Butler': 38, ' Vitaa': 39, 'Russell Crowe': 40, 'Nina Hoss': 41, 'Megan Suri': 42, 'Patrick Wilson': 43, 'Yuriy Borisov': 44, 'Nicolas Cage': 45, 'Laura Paredes': 46, 'Doona Bae': 47, 'Alice Pol': 48, 'Keri Rus

,nationalite,studio,timing,title,titre_original,entrees_fr,num_actors,actor_1,actor_2,actor_3,num_director,director_1,director_2,director_3,popularity_actor_1,popularity_actor_2,popularity_actor_3
1,Allemagne,Tandem,99,La Salle des profs,Das Lehrerzimmer,90612,3,Leonie Benesch,Michael Klammer,Rafael Stachowiak,3,İlker Çatak,İlker Çatak,Johannes Duncker,123.0,501.0,895.0
2,Italie,Universal Pictures International France,118,Il reste encore demain,C'è ancora domani,145340,3,Paola Cortellesi,Valerio Mastandrea,Romana Maggiora Vergano,3,Paola Cortellesi,Paola Cortellesi,Furio Andreotti,124.0,502.0,896.0
5,U.S.A.,Warner Bros. France,166,Dune : Deuxième Partie,Dune: Part Two,1383999,3,Timothée Chalamet,Zendaya,Rebecca Ferguson,3,Denis Villeneuve,Denis Villeneuve,Jon Spaihts,125.0,503.0,897.0
7,Grande-Bretagne,SND,109,Une vie,One Life,456390,3,Anthony Hopkins,Johnny Flynn,Helena Bonham Carter,3,James Hawes,Lucinda Coxon,Nick Drake,126.0,504.0,898.0
12,Grande-Bretagne,StudioCanal,100,Scandaleusement vôtre,Wicked Little Letters,51921,3,Olivia Colman,Jessie Buckley,Anjana Vasan,2,Thea Sharrock,Jonny Sweet,None,127.0,81.0,899.0


In [74]:
#Donner une cote aux directors
from collections import Counter
import numpy as np

# Combiner les listes d'acteurs dans une seule liste
all_directors = merged_data['director_1'].tolist() + merged_data['director_2'].tolist() + merged_data['director_3'].tolist()

# Utiliser Counter pour compter le nombre d'occurrences de chaque acteur
director_counts = Counter([director for director in all_directors if isinstance(director, str)])

# Trier les acteurs par nombre d'occurrences (le plus fréquent en premier)
sorted_directors = sorted(director_counts.items(), key=lambda x: x[1], reverse=True)

# Créer un dictionnaire pour stocker les scores des acteurs
director_scores = {}

# Assigner des scores en fonction du classement
for i, (director, count) in enumerate(sorted_directors, start=1):
    director_scores[director] = i

print(director_scores)
print(len(director_scores))

# Définir une fonction pour mapper les directors à leur score de popularité
def map_director_to_popularity(director, director_scores):
    if isinstance(director, str) and director in director_scores:
        return director_scores[director]
    else:
        return np.nan

# Ajouter la fonctionnalité popularity_director_1
merged_data['popularity_director_1'] = merged_data['director_1'].map(lambda x: map_director_to_popularity(x, director_scores))

# Ajouter la fonctionnalité popularity_director_2
merged_data['popularity_director_2'] = merged_data['director_2'].map(lambda x: map_director_to_popularity(x, director_scores))

# Ajouter la fonctionnalité popularity_director_3
merged_data['popularity_director_1'] = merged_data['director_3'].map(lambda x: map_director_to_popularity(x, director_scores))

# Afficher les premières lignes du DataFrame pour vérification
display(merged_data.head())

{'Hong Sang-Soo': 1, 'Ryūsuke Hamaguchi': 2, 'Ali Asgari': 3, 'Cal Brunker': 4, 'Christian Petzold': 5, 'Destin Daniel Cretton': 6, 'Adilkhan Yerzhanov': 7, 'Laura Citarella': 8, 'Kirill Serebrennikov': 9, 'Radu Jude': 10, 'Amanda Sthers': 11, 'Kôji Fukada': 12, 'Ahmad Bahrami': 13, 'Haruo Sotozaki': 14, 'Agnieszka Holland': 15, 'Hirokazu Kore-eda': 16, 'Scott Beck': 17, 'Justin Lin': 18, 'Eskil Vogt': 19, 'Evan Spiliotopoulos': 20, 'David Leslie Johnson-McGoldrick': 21, 'Bob Barlen': 22, 'İlker Çatak': 23, 'Paola Cortellesi': 24, 'Denis Villeneuve': 25, 'Jonathan Glazer': 26, 'Andrew Haigh': 27, 'Terence Davies': 28, 'Tian Xiaopeng': 29, 'Amjad Al Rasheed': 30, 'Jason Yu': 31, 'DK Welchman': 32, 'Claudia Huaiquimilla': 33, 'Veerle Baetens': 34, 'Jim Capobianco': 35, 'Kenneth Branagh': 36, 'Matt Carter (II)': 37, 'Estibaliz Urresola Solaguren': 38, 'Celine Song': 39, 'Sean Durkin': 40, 'Timm Kröger': 41, 'Matteo Garrone': 42, 'Marco Bellocchio': 43, 'Ridley Scott': 44, 'Lucy Izzard': 4

,nationalite,studio,timing,title,titre_original,entrees_fr,num_actors,actor_1,actor_2,actor_3,num_director,director_1,director_2,director_3,popularity_actor_1,popularity_actor_2,popularity_actor_3,popularity_director_1,popularity_director_2
1,Allemagne,Tandem,99,La Salle des profs,Das Lehrerzimmer,90612,3,Leonie Benesch,Michael Klammer,Rafael Stachowiak,3,İlker Çatak,İlker Çatak,Johannes Duncker,123.0,501.0,895.0,676.0,23.0
2,Italie,Universal Pictures International France,118,Il reste encore demain,C'è ancora domani,145340,3,Paola Cortellesi,Valerio Mastandrea,Romana Maggiora Vergano,3,Paola Cortellesi,Paola Cortellesi,Furio Andreotti,124.0,502.0,896.0,677.0,24.0
5,U.S.A.,Warner Bros. France,166,Dune : Deuxième Partie,Dune: Part Two,1383999,3,Timothée Chalamet,Zendaya,Rebecca Ferguson,3,Denis Villeneuve,Denis Villeneuve,Jon Spaihts,125.0,503.0,897.0,678.0,25.0
7,Grande-Bretagne,SND,109,Une vie,One Life,456390,3,Anthony Hopkins,Johnny Flynn,Helena Bonham Carter,3,James Hawes,Lucinda Coxon,Nick Drake,126.0,504.0,898.0,679.0,486.0
12,Grande-Bretagne,StudioCanal,100,Scandaleusement vôtre,Wicked Little Letters,51921,3,Olivia Colman,Jessie Buckley,Anjana Vasan,2,Thea Sharrock,Jonny Sweet,None,127.0,81.0,899.0,NaN,487.0


In [75]:
# classer les nationalite en continent
import pandas as pd

# Créer un dictionnaire de correspondance entre les nationalités et les continents
nationality_to_continent = {
    'Allemagne': 'Europe',
    'Italie': 'Europe',
    'U.S.A.': 'Amérique du Nord',
    'Grande-Bretagne': 'Europe',
    'Irlande': 'Europe',
    'Chine': 'Asie',
    'Jordanie': 'Asie',
    'Iran': 'Asie',
    'Corée du Sud': 'Asie',
    'Pologne': 'Europe',
    'Chili': 'Amérique du Sud',
    'Belgique': 'Europe',
    'Japon': 'Asie',
    'Espagne': 'Europe',
    'Finlande': 'Europe',
    'Danemark': 'Europe',
    'Grèce': 'Europe',
    'Turquie': 'Asie',
    'Suède': 'Europe',
    'Australie': 'Océanie',
    'Mongolie': 'Asie',
    'Russie': 'Europe',
    'Yemen': 'Asie',
    'France': 'Europe',
    'Suisse': 'Europe',
    'Canada': 'Amérique du Nord',
    'Indonésie': 'Asie',
    'Ukraine': 'Europe',
    'Norvège': 'Europe',
    'kazakhstan': 'Asie',
    'Soudan': 'Afrique',
    'Argentine': 'Amérique du Sud',
    'Géorgie': 'Europe',
    'Bhoutan': 'Asie',
    'Lituanie': 'Europe',
    'Brésil': 'Amérique du Sud',
    'Roumanie': 'Europe',
    'Hong-Kong': 'Asie',
    'Egypte': 'Afrique',
    'Bosnie-Herzégovine': 'Europe',
    'Tunisie': 'Afrique',
    'Hongrie': 'Europe',
    'Guatemala': 'Amérique du Nord',
    'Islande': 'Europe',
    'Slovaquie': 'Europe',
    'Pérou': 'Amérique du Sud',
    'Portugal': 'Europe',
    'Pays-Bas': 'Europe',
    'Colombie': 'Amérique du Sud',
    'Luxembourg': 'Europe',
    'Autriche': 'Europe',
    'Palestine': 'Asie',
    'République tchèque': 'Europe',
    'Macédoine': 'Europe',
    'Inde': 'Asie',
    'Philippines': 'Asie',
    'Kosovo': 'Europe',
    'Bolivie': 'Amérique du Sud',
    'Israël': 'Asie',
    'Liban': 'Asie',
    'Taïwan': 'Asie',
    'Algérie': 'Afrique',
    'Cambodge': 'Asie',
    'Uruguay': 'Amérique du Sud',
    'Thaïlande': 'Asie',
    'Lésotho': 'Afrique',
    'Costa Rica': 'Amérique du Nord',
    'Mexique': 'Amérique du Nord'
}

# Définir une fonction pour mapper les nationalités vers les continents
def map_nationality_to_continent(nationality):
    nationality_cleaned = nationality.strip()  # Supprimer les espaces blancs supplémentaires
    if nationality_cleaned in nationality_to_continent:
        return nationality_to_continent[nationality_cleaned]
    else:
        return 'Autre'

# Appliquer la fonction pour créer la nouvelle colonne 'continents'
merged_data['continents'] = merged_data['nationalite'].apply(map_nationality_to_continent)

# Afficher les premières lignes du DataFrame pour vérification
display(merged_data)

,nationalite,studio,timing,title,titre_original,entrees_fr,num_actors,actor_1,actor_2,actor_3,num_director,director_1,director_2,director_3,popularity_actor_1,popularity_actor_2,popularity_actor_3,popularity_director_1,popularity_director_2,continents
1,Allemagne,Tandem,99,La Salle des profs,Das Lehrerzimmer,90612,3,Leonie Benesch,Michael Klammer,Rafael Stachowiak,3,İlker Çatak,İlker Çatak,Johannes Duncker,123.0,501.0,895.0,676.0,23.0,Europe
2,Italie,Universal Pictures International France,118,Il reste encore demain,C'è ancora domani,145340,3,Paola Cortellesi,Valerio Mastandrea,Romana Maggiora Vergano,3,Paola Cortellesi,Paola Cortellesi,Furio Andreotti,124.0,502.0,896.0,677.0,24.0,Europe
5,U.S.A.,Warner Bros. France,166,Dune : Deuxième Partie,Dune: Part Two,1383999,3,Timothée Chalamet,Zendaya,Rebecca Ferguson,3,Denis Villeneuve,Denis Villeneuve,Jon Spaihts,125.0,503.0,897.0,678.0,25.0,Amérique du Nord
7,Grande-Bretagne,SND,109,Une vie,One Life,456390,3,Anthony Hopkins,Johnny Flynn,Helena Bonham Carter,3,James Hawes,Lucinda Coxon,Nick Drake,126.0,504.0,898.0,679.0,486.0,Europe
12,Grande-Bretagne,StudioCanal,100,Scandaleusement vôtre,Wicked Little Letters,51921,3,Olivia Colman,Jessie Buckley,Anjana Vasan,2,Thea Sharrock,Jonny Sweet,None,127.0,81.0,899.0,NaN,487.0,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4703,U.S.A.,\n385 entrées\n,60,"Barbie : Grande Ville, Grands Rêves","Barbie: Big City, Big Dreams",391,3,America Young,Greg Chun,Kirsten Day,3,Scott Pleydell-Pearce,Christopher Keenan,Kate Splaine,496.0,890.0,1303.0,849.0,674.0,Amérique du Nord
4812,Chine,ASC Distribution,104,Grand frère,Wisdom Tooth,414,3,Lu Celeste,Wu Xiaoliang,Jiajia Wang,2,Liang Ming,Liang Ming,None,497.0,891.0,1304.0,NaN,303.0,Asie
4948,Autriche,Les Films du Losange,102,Oskar et Lily,Ein bisschen bleiben wir noch,1324,3,Leopold Pallua,Rosa Zant,Anna Fenderl,2,Arash T. Riahi,Arash T. Riahi,None,498.0,892.0,1305.0,NaN,304.0,Europe
4988,France,Cinéma Saint-André des Arts,88,Le Voyage d'Ana,El Viaje de Ana,108,3,Astrid Adverbe,Dimitra Kontou,Maritza Gonzalez,2,Pamela Varela,Pamela Varela,None,499.0,893.0,1306.0,NaN,305.0,Europe


In [76]:
#exportation du dataset netoye
display(merged_data)

merged_data.to_csv("datasets/allocine_cleaned.csv", index=None)

merged_data.to_pickle("datasets/dataset.pkl")

,nationalite,studio,timing,title,titre_original,entrees_fr,num_actors,actor_1,actor_2,actor_3,num_director,director_1,director_2,director_3,popularity_actor_1,popularity_actor_2,popularity_actor_3,popularity_director_1,popularity_director_2,continents
1,Allemagne,Tandem,99,La Salle des profs,Das Lehrerzimmer,90612,3,Leonie Benesch,Michael Klammer,Rafael Stachowiak,3,İlker Çatak,İlker Çatak,Johannes Duncker,123.0,501.0,895.0,676.0,23.0,Europe
2,Italie,Universal Pictures International France,118,Il reste encore demain,C'è ancora domani,145340,3,Paola Cortellesi,Valerio Mastandrea,Romana Maggiora Vergano,3,Paola Cortellesi,Paola Cortellesi,Furio Andreotti,124.0,502.0,896.0,677.0,24.0,Europe
5,U.S.A.,Warner Bros. France,166,Dune : Deuxième Partie,Dune: Part Two,1383999,3,Timothée Chalamet,Zendaya,Rebecca Ferguson,3,Denis Villeneuve,Denis Villeneuve,Jon Spaihts,125.0,503.0,897.0,678.0,25.0,Amérique du Nord
7,Grande-Bretagne,SND,109,Une vie,One Life,456390,3,Anthony Hopkins,Johnny Flynn,Helena Bonham Carter,3,James Hawes,Lucinda Coxon,Nick Drake,126.0,504.0,898.0,679.0,486.0,Europe
12,Grande-Bretagne,StudioCanal,100,Scandaleusement vôtre,Wicked Little Letters,51921,3,Olivia Colman,Jessie Buckley,Anjana Vasan,2,Thea Sharrock,Jonny Sweet,None,127.0,81.0,899.0,NaN,487.0,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4703,U.S.A.,\n385 entrées\n,60,"Barbie : Grande Ville, Grands Rêves","Barbie: Big City, Big Dreams",391,3,America Young,Greg Chun,Kirsten Day,3,Scott Pleydell-Pearce,Christopher Keenan,Kate Splaine,496.0,890.0,1303.0,849.0,674.0,Amérique du Nord
4812,Chine,ASC Distribution,104,Grand frère,Wisdom Tooth,414,3,Lu Celeste,Wu Xiaoliang,Jiajia Wang,2,Liang Ming,Liang Ming,None,497.0,891.0,1304.0,NaN,303.0,Asie
4948,Autriche,Les Films du Losange,102,Oskar et Lily,Ein bisschen bleiben wir noch,1324,3,Leopold Pallua,Rosa Zant,Anna Fenderl,2,Arash T. Riahi,Arash T. Riahi,None,498.0,892.0,1305.0,NaN,304.0,Europe
4988,France,Cinéma Saint-André des Arts,88,Le Voyage d'Ana,El Viaje de Ana,108,3,Astrid Adverbe,Dimitra Kontou,Maritza Gonzalez,2,Pamela Varela,Pamela Varela,None,499.0,893.0,1306.0,NaN,305.0,Europe
